#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 2:  Link Analysis -- PageRank + SEO

### 100 points [5% of your final grade]

### Due: Tuesday, February 21, 2017 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test PageRank over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 24, 2017 at 11:59pm.

# Part 1: PageRank (70 points)

## A Twitter-Mentioning Graph

In this assignment, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their mentions of other Twitter users (so user = node, mention of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is mentioned by other users is more "impactful". 

Here is a toy example. Suppose you are given the following four tweets:

* **userID**: diane, **text**: "@bob Howdy!"
* **userID**: charlie, **text**: "Welcome @bob and @alice!"
* **userID**: bob, **text**: "Hi @charlie and @alice!"
* **userID**: alice, **text**: "Howdy!"

There are four short tweets generated by four users. The @mentions between users form a directed graph with four nodes and five edges. E.g., the "diane" node has a directed edge to the "bob" node. Note that a retweet also contain the "@", so it should be counted as a mention as well.

You should build a graph by parsing the tweets in the file we provide called *pagerank.json*.

**Notes:**

* The edges are binary and directed. If Bob mentions Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user mentions herself, ignore it.
* Correctly parsing @mentions in a tweet is error-prone. Use the entities field.
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [1]:
# Here define your function for building the graph by parsing the input file of tweets

import json
from pprint import pprint

''' Input : filename of the tweets
    Output : graph and user_id and name/screen_name mapping    
'''
def parse_json_file(filename,userid_names_dict , userid_screennames_dict ):
    userid_adjancency_graph = {}
    for line in open(filename, 'r'): # Reading each tweet one by one
        tweets = json.loads(line)
        # Reading the userid of the tweet, and also checking the mentions of the tweet : Here adding only the screen_name
        userid_screennames_dict.setdefault(tweets['user']['id'], set()).add(tweets['user']['screen_name'])

        #Checking the Mentions of all the tweets and storing the user mentioned to make the graph and also storing the 
        # mentioned users name and screen_name.
        for mentions in tweets['entities']['user_mentions']:
            userid_names_dict.setdefault(mentions['id'], set()).add(mentions['name'])      # Storing the Actual Name of user
            userid_screennames_dict.setdefault(mentions['id'], set()).add(mentions['screen_name']) # Storing the Screen Name of user
            if(mentions['id'] != tweets['user']['id']):
                userid_adjancency_graph.setdefault(tweets['user']['id'],[set() for _ in xrange(2)])[0].add(mentions['id']) # making the adjanceny graph
                userid_adjancency_graph.setdefault(mentions['id'],[set() for _ in xrange(2)])[1].add(tweets['user']['id']) 
    return userid_adjancency_graph


In [16]:
# Now We already have the graph and the user_id and screen_name/name mapping
def get_graph_from_dict(filename):
    userid_names_dict = {}
    userid_screennames_dict = {}
    userid_adjancency_graph = parse_json_file(filename,userid_names_dict, userid_screennames_dict)
    print 'Number of Nodes: ' , len(userid_adjancency_graph)
    num_edges = 0
    for node in userid_adjancency_graph:
        num_edges += len(userid_adjancency_graph[node][0])
    print 'Number of Edges: ', num_edges
    return userid_adjancency_graph

graph = get_graph_from_dict('pagerank.json')

Number of Nodes:  16430
Number of Edges:  24256


In [14]:
# Getting the graph using the networks library
import networkx as net

# Generate the graph using the network library from the ditionary already generated

def generate_network(graph):
    num_nodes = len(graph)
    network_graph = net.DiGraph()
    for node in graph:

        outgoing_nodes = graph[node][0]

        for nei_nodes in outgoing_nodes:
                network_graph.add_edge(node , nei_nodes)
   
    print 'Number of Nodes: ', len(network_graph.nodes())
    print 'Number of Edges: ', len(network_graph.edges())
    return network_graph

net_graph = generate_network(graph)



Number of Nodes:  16430
Number of Edges:  24256


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## PageRank Implementation

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never mentioned and does not mention anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions (Numpy 1.12.0; Scipy 0.18.1).
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

The Stopping Condition for the iterations:

I am taking the stopping criterion of this form:
Sum over all pages/Nodes (PR(A)_i+1 - PR(A)_i)^2 < N*epsilon^2, 
where N is the number of pages, i denotes the iteration and epsilon is some small fixed number, here epsilon = 0.00001.

https://www.webmasterworld.com/forum3/25867.htm


I also tried the suggested stopping condition by Cav, which was at each iteration check if the top ten results are changing, if they are then keep iterating, if not then probably we may stop. But when i ran that it makes the system to everytime sort the results. Which is basically O(nlogn) operation, whereas if we increase the number of iteration on all the sets it will still be O(N) operation. 

In [17]:
########################################### PAGE RANK #####################################################################
# https://www.webmasterworld.com/forum3/25867.htm : The Stopping Condition for the iterations
# Normally, one would take a stopping criteria of the following form: 
# sum over all pages (PR(A)_i+1 - PR(A)_i)^2 < N*epsilon^2, 
# where N is the number of pages, i denotes the iteration and epsilon is some small fixed number.

############################################Self Defined GRAPH##############################################################

import copy

'''
Input -> graph in form of adjaceny List
d = damping factor
epsilon - to stopping criteria

Output: 10 ranking list

'''

def PageRanker(graph, d , epsilon):
    num_nodes = len(graph)
    initial_weight =  ((1-float(d)) /(float)(num_nodes))  # Precomputed Fixed Weight
    pagerank = {}
    
    initial_rank = (1/float (num_nodes))   # Precomputed Initial Rank
    
    for nodes in graph:
        pagerank[nodes] = initial_rank
    
    for cnt in range(500):     # Top Number of Iterations
        pagerank_prev = copy.deepcopy(pagerank)    # Deep Copy to store the old values
        
        for nodes in graph:
            sum_rank = 0;
            
            in_links = graph[nodes][1]    # Getting the inlinks of a node to calculate the rank
            
            for n in in_links:
                num_outgoing_links = len(graph[n][0])   # Since we need to distribute the weights among all outgoing links.
                if num_outgoing_links > 0:
                    sum_rank +=  (1/(float(num_outgoing_links))) * pagerank_prev[n]
                    
            pagerank[nodes] = initial_weight + d * sum_rank   # Ranking updation
        sum_error = 0
        for nodes in graph:
            sum_error += pow(pagerank[nodes] - pagerank_prev[nodes], 2)
         
        if sum_error < num_nodes * pow(epsilon, 2):        # Stopping Condition
            break
            

    # Rank Normalization 
    normalization_factor = 0
    for node in pagerank:
        normalization_factor = normalization_factor + pagerank[node]
    
    for node in pagerank:
        pagerank[node] = pagerank[node]/normalization_factor
    
    # sort the ranks     
    sorted_ranks = sorted(pagerank.items(), key = operator.itemgetter(1), reverse = True)
    sorted_ranks = sorted_ranks[:10]
    
    print 'Results: '
    for node in sorted_ranks:
        print 'User-Id: ' , node[0] , '\t Score: ' , node[1]
    print 'Total Iteration Requred: ', cnt


In [18]:
# Now let's call your function on the graph you've built. Output the results.

# As we have already build the graph in the second cell. We will just use the same

PageRanker(graph, 0.9, 0.000001)

Results: 
User-Id:  158314798 	 Score:  0.0076884614137
User-Id:  181561712 	 Score:  0.00649836514478
User-Id:  209708391 	 Score:  0.0060514253914
User-Id:  72064417 	 Score:  0.00551595034247
User-Id:  105119490 	 Score:  0.00506482318195
User-Id:  14268057 	 Score:  0.00466132073579
User-Id:  379961664 	 Score:  0.00440795277942
User-Id:  391037985 	 Score:  0.00424082737743
User-Id:  153074065 	 Score:  0.00423748800169
User-Id:  313525912 	 Score:  0.00407948802252
Total Iteration Requred:  21


In [19]:
#################################PAGE RANK ON NETWORK GRAPHS############################################################
import operator
import copy
   
def PageRanker_Network(graph, d ,epsilon):
    num_nodes = len(graph.nodes())
    initial_weight =  ((1-float(d)) /(float)(num_nodes)) 
    pagerank = {}
    
    initial_rank = (1/float (num_nodes))
    
    for nodes in graph.nodes():
        pagerank[nodes] = initial_rank
    
    for cnt in range(500):     # Top number of iterations
        pagerank_prev = copy.deepcopy(pagerank)
        for nodes in graph.nodes():
            sum_rank = 0;
            
            in_links = graph.in_edges(nodes)
            
            for n in in_links:
                num_incom_links = len(graph.out_edges(n[0]))
                if num_incom_links > 0:
                    sum_rank +=  (1/(float(num_incom_links))) * pagerank_prev[n[0]]
           
            pagerank[nodes] = initial_weight + d * sum_rank   # Ranking updation
        sum_error = 0
        for nodes in graph:
            sum_error += pow(pagerank[nodes] - pagerank_prev[nodes], 2)
         
        if sum_error < num_nodes * pow(epsilon, 2):        # Stopping Condition
            break
            

    # Rank Normalization 
    normalization_factor = 0
    for node in pagerank:
        normalization_factor = normalization_factor + pagerank[node]
    
    for node in pagerank:
        pagerank[node] = pagerank[node]/normalization_factor
    
    # sort the ranks     
    sorted_ranks = sorted(pagerank.items(), key = operator.itemgetter(1), reverse = True)
    sorted_ranks = sorted_ranks[:10]
    
    print 'Results: '
    for node in sorted_ranks:
        print 'User-Id: ' , node[0] , '\t Score: ' , node[1]
    print 'Total Iteration Requred: ', cnt

    

PageRanker_Network(net_graph,0.9, 0.000001)


Results: 
User-Id:  158314798 	 Score:  0.0076884614137
User-Id:  181561712 	 Score:  0.00649836514478
User-Id:  209708391 	 Score:  0.0060514253914
User-Id:  72064417 	 Score:  0.00551595034247
User-Id:  105119490 	 Score:  0.00506482318195
User-Id:  14268057 	 Score:  0.00466132073579
User-Id:  379961664 	 Score:  0.00440795277942
User-Id:  391037985 	 Score:  0.00424082737743
User-Id:  153074065 	 Score:  0.00423748800169
User-Id:  313525912 	 Score:  0.00407948802252
Total Iteration Requred:  21


# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **awcv9kjlh scwrlkjf4e** --- two terms, lower case, no quote. As of today (Feb 7, 2017), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

http://people.tamu.edu/~adil.hamid/awcv9kjlh%20scwrlkjf4e.html

What's your strategy? (2-4 paragraphs)

I followed the strategy of making my webpage available for indexing at multiple places.

First of all i created a page on my TAMU website i.e http://people.tamu.edu/~adil.hamid/ with the keyword as new page.
Now to make it indexed over the web i needed to mention to link this page with other well known trusted pages like facebook, wikipedia etc. So, i mentioned the links to few of the well know dynamic pages. 

Second, strategy i followed is that i used few keywords or buzz words in the meta data of the html pages. So that it gets indexed. Apart from it, i didn't include too much text related to query because search engines may penalize for that.

Thirdly, i made github.io page which pointed to my page. Since github.io is considered to be good and trusted webpage, so it will make my webpage get indexed more quickly and add some weights to its ranking. 

I also, made some blogspot page and page on heroku app, so that they all point to my website i.e. http://people.tamu.edu/~adil.hamid/awcv9kjlh%20scwrlkjf4e.html. It helped to get it indexed.

But all these pages being static means that the rate of crawling next time is very low, so i updated the root website on my LinkedIn page, as the LinkedIn pages are highly dynamic, so it might lead to my page getting indexed more quickly and often. I practically saw that when my page got indexed on google, even after i changed the content it was still showing the old results, which were kind of expected, that is why we must link our website at locations/web pages which get indexed very frequently.

Finally, I also made a video on youtube to introduce the words and linked my webpage out there, so that it gets indexed.

And finally my page got indexed by the Google.

I will still work on the ranking of my webpage in Google.

